# NUCLEAR PASTA IN LAMMPS

## Natural units (some useful conversions):
Fundamental quantities: $\hbar$ (action), $c$ (speed of light), $eV$.
$$1 c = 2.99792458 \times 10^10 cm/s\\
1 \hbar = 1.05457266\times 10^{-27}g~cm^2/s\\
1 eV = 1.60217733 \times 10^{-12} g~cm^2/s^2\\
1 s = 1.51926689 \times 10^{15} \hbar/eV\\
1 cm = 5.06772886 \times 10^{4} \hbar c /eV\\
1 g = 5.60958616 \times 10^{32} eV/c^2\\
\hbar c = 197.327053 MeV ~ fm\\
e^2 = [137.03985]^{-1} \hbar c$$

## Units
$$
[mass]= eV/c^2\\
[time]=\hbar/eV\\
[length]=\hbar c/eV\\
[momentum]=eV/c\\
[force]=eV^2/\hbar c\\
[pressure]=eV^4 / \hbar^3 c^3\\
[charge]=\hbar c
$$

## LAMMPS metal units
- [mass] = grams/mole
- [distance] = Angstroms
- [time] = picoseconds
- [energy] = eV
- [velocity] = Angstroms/picosecond
- [force] = eV/Angstrom
- [torque] = eV
- [temperature] = Kelvin
- [pressure] = bars
- [dynamic viscosity] = Poise
- [charge] = multiple of electron charge (1.0 is a proton)
- [dipole] = charge*Angstroms
- [electric field] = volts/Angstrom
- [density] = gram/cm^dim

## Conversion factors between Natural units and Metal LAMMPS units
- Distance $ 1~ A = 10^{5}~ fm $ and $1~ fm = 10^{-5}~ A$
- Time $1~ ps = 2.9978 \times 10^{11}~ fm/c$ and $1~fm/c = 3.33 \times 10^{-12}~ ps$
- Temperature $1~ eV = 1.1604 \times 10^{4}~K$, $1~ MeV = 1.1604\times 10^{10}~K$, and $1~K=8.617343 \times 10^{-5} ~eV$
- Boltzmann constant $k_B=8.617343 \times 10^{-5}~eV/K$(LAMMPS) and $k_B=1$(in natural units)
- Mass $M_{proton}= 1~ uma$ (LAMMPS) and $M_{proton}=938.272~ MeV /c^2$
- Mass $M_{neutron}= 1.00137~ uma$ (LAMMPS) and  $M_{neutron}=939.56~ MeV /c^2$

## Set up simulation

In [5]:
import numpy as np
# Number of particles
n_part = 6859
# Reduced density (Number of particles/fm^3)
density_part = 0.01
sim_box_length = np.cbrt(n_part/density_part) #cube root (fm)
sim_box_length = sim_box_length * 1e-5 # sim_box_lenght in A
print('Simulation box length: {0:2.2e} A'.format(sim_box_length))

Simulation box length: 8.82e-04 A


* Simulation box 
$$
L^3 = N/\rho^{*}
$$

In [6]:
# Particle per length unit
n_unit_cell = np.cbrt(n_part)
print('Number of unit cells: {0:2.0f}'.format(n_unit_cell))
lattice_const = sim_box_length / n_unit_cell
print('Lattice constant: {0:2.2e} A'.format(lattice_const))

Number of unit cells: 19
Lattice constant: 4.64e-05 A


In [7]:
# Mass
proton_frac = 0.2 # Proton fraction 
print('Number of protons: {0:5.1f}'.format(n_part * proton_frac))
print('Number of neutrons: {0:5.1f}'.format(n_part * (1-proton_frac)))
m_proton=1.0
m_neutron=1.00137

Number of protons: 1371.8
Number of neutrons: 5487.2


In [8]:
# Time and temperature
time_step = 6.66e-12 # 6.66e-12 ps = 2 fm/c
print('Time step: {0:2.2e}'.format(time_step))
temp = 1.1604e10 # temperature 1.16K = 1 MeV
print('Temperature: {0:2.2e}'.format(temp))

Time step: 6.66e-12
Temperature: 1.16e+10


## Potential
$$
V(i,j)=-A e^{-r^2/\Lambda} - [B + C \tau_z(i) \tau_z(j)] e^{-r^2/2 \Lambda} + \frac{q_e^2}{r} e^{-r/ \kappa} \tau_p(i) \tau_p(j)
$$
where $\tau_z$ is the nucleon isospin projection ($\tau_z=+1$ for protons and $\tau_z=-1$ for neutrons).

For the screened Coulomb interaction $\tau_p=(1+\tau_z)/2$. $q_e$ is the electron charge. In natural units,
$q_e^2/4 \pi \epsilon_0=1.43996\ MeV\ fm = 14.399\ eV\ A$

This can be done using the pair/style

`lmp.command("pair_style hybrid/overlay gauss {0} gauss {0} yukawa {1} {0}".format(cutoff,kappa))`

In [ ]:
a = -110e6
b1 = 2e6 # For proton-proton or neutron-neutron interaction
b2 = 50e6 # For proton-neutron interaction
c = 14.399 # factor e^2 in Yukawa potential, just for proton-proton interaction
Lambda = 1.25e-10 # for gaussian potentials
kappa = 10e-5 # For Yukawa, Thomas-Fermi lenght
cutoff = 2e-4 # cutoff distance for potentials
print('Cutoff distance: {0:2.2e}'.format(cutoff))
random_seed = 1234

lmp.command("pair_style hybrid/overlay gauss {0} gauss {0} yukawa {1} {0}".format(cutoff,kappa))
# Pair coefficients for neutron-neutron interaction
lmp.command("pair_coeff 1 1 gauss 1 {0} {1}".format(a,1.0/Lambda))
lmp.command("pair_coeff 1 1 gauss 2 {0} {1}".format(b1,0.5/Lambda))
lmp.command("pair_coeff 1 1 yukawa {0}".format(0.0))
# Pair coefficients for neutron-proton interaction
lmp.command("pair_coeff 1 2 gauss 1 {0} {1}".format(a,1.0/Lambda))
lmp.command("pair_coeff 1 2 gauss 2 {0} {1}".format(b2,0.5/Lambda))
lmp.command("pair_coeff 1 2 yukawa {0}".format(0.0))
# Pair coefficients for proton-proton interaction
lmp.command("pair_coeff 2 2 gauss 1 {0} {1}".format(a,1.0/Lambda))
lmp.command("pair_coeff 2 2 gauss 2 {0} {1}".format(b1,0.5/Lambda))
lmp.command("pair_coeff 2 2 yukawa {0}".format(c))